In [1]:
import sys; sys.path.append("../")
import pandas as pd

from lib.explore import show_unique
from lib.columns import rearrange_personel_history_columns, rearrange_personel_columns
from lib.match import (
    ThresholdClassifier, ColumnsIndex, StringSimilarity,
    DateSimilarity, match_records, print_match_result
)
from lib.uid import gen_uid
from clean.new_orleans_harbor_pd_cprr import clean as clean_cprr
from clean.new_orleans_harbor_pd_pprr import clean_personel

In [2]:
cprr = clean_cprr()
cprr

,badge_no,sex,department_desc,rank_desc,rule_code,paragraph_code,recommended_action,action,incident_type,tracking_number,...,occur_month,occur_day,receive_year,receive_month,receive_day,investigation_complete_year,investigation_complete_month,investigation_complete_day,agency,data_production_year
0,HP-56,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2015-3,...,7,5,2015,7,9,2015,8,4,New Orleans Harbor PD,2020
1,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2015-2,...,3,22,2015,4,1,2015,6,29,New Orleans Harbor PD,2020
2,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2016-3,...,6,25,2016,6,27,2016,7,29,New Orleans Harbor PD,2020
3,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2016-5,...,11,20,2016,12,21,2017,1,11,New Orleans Harbor PD,2020
4,HP-33,male,bicycle division,corporal,1.3,2,none,,use of force,2017-7,...,7,15,2017,7,15,2017,8,29,New Orleans Harbor PD,2020
5,HP-56,male,,police officer 2-a,2,3,counselling memorrandum,counselling memmorandum,discourtesy,2017-1,...,12,24,2017,1,10,2017,2,20,New Orleans Harbor PD,2020


In [3]:
pprr = clean_personel()
pprr

,first_name,last_name,rank_desc,hourly_salary,middle_initial,term_year,term_month,term_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,uid,agency,data_production_year
0,casey,adams,police sergeant--port,17.00,m,2010,2,28,1980,1,31,1997,4,14,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
1,casey,adams,police sergeant--port,17.68,m,2010,2,28,1980,1,31,2003,12,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
2,casey,adams,police sergeant--port,18.39,m,2010,2,28,1980,1,31,2004,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
3,casey,adams,police sergeant--port,19.13,m,2010,2,28,1980,1,31,2005,8,10,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
4,casey,adams,police sergeant-a,19.13,m,2010,2,28,1980,1,31,2005,8,29,0e62eddff885ed636a82cd726b390596,New Orleans Harbor PD,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,brianne,waters,police officer 3-a,21.60,m,2020,5,11,2001,3,5,2015,10,1,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
911,brianne,waters,police sergeant-a,24.34,m,2020,5,11,2001,3,5,2018,5,14,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
912,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020
913,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,b1014901a4b7ddee9ecd880405127d81,New Orleans Harbor PD,2020


In [4]:
# cprr[["first_name", "last_name", "appoint_year", "appoint_month", "appoint_day"]]
cprr_appoint_dates = cprr[["appoint_year", "appoint_month", "appoint_day"]]
cprr_appoint_dates.columns = ["year", "month", "day"]
cprr.loc[:, "hire_date"] = pd.to_datetime(cprr_appoint_dates)
cprr = gen_uid(cprr, ["first_name", "last_name", "appoint_year", "appoint_month", "appoint_day"], "mid")

pprr_hire_dates = pprr[["hire_year", "hire_month", "hire_day"]]
pprr_hire_dates.columns = ["year", "month", "day"]
pprr.loc[:, "hire_date"] = pd.to_datetime(pprr_hire_dates)

In [5]:
dfa = cprr[["mid", "first_name", "last_name", "hire_date"]].drop_duplicates()
dfa = dfa.set_index("mid", drop=True)
dfb = pprr[["uid", "first_name", "last_name", "hire_date"]].drop_duplicates()
dfb = dfb.set_index("uid", drop=True)
matches, potential_matches, non_matches = match_records(
    ColumnsIndex(["first_name"]),
    ThresholdClassifier(
        fields={
            "last_name": StringSimilarity(),
            "hire_date": DateSimilarity()
        },
        thresholds=[1, 0.5]
    ),
    dfa,
    dfb
)
print_match_result(dfa, dfb, matches, potential_matches, non_matches)

*** MATCHES ***
                                 first_name last_name  hire_date
ee52e3e61460449f9f78c04324d676c5      bryan     arena 2013-10-21
e6278fa24d4f3a03c7b1fc41f07dd6de      bryan     arena 2013-10-21

*** POTENTIAL MATCHES ***
                                 first_name last_name  hire_date
eda7bfbaa2d650b819233c6eb0e8e44f      lamus     lundy 2011-02-28
cac9967e392f4edf4da41c4e478f96b5      lamus  lundy jr 2011-02-28

                                 first_name last_name  hire_date
34a7cb7ba8141fd146dea1a9da9b47ed      lamus     lundy 2019-02-28
cac9967e392f4edf4da41c4e478f96b5      lamus  lundy jr 2011-02-28

                                 first_name last_name  hire_date
dd88a725950340ca1286ed3e719ab7a5     howard  dennison 1998-02-19
2e448f6aba55cd31091991480f1af231     howard  dennison 1998-03-02



In [6]:
mid_to_uid_d = dict(matches + potential_matches)
cprr.loc[:, "uid"] = cprr["mid"].map(lambda v: mid_to_uid_d[v])
cprr = cprr.drop(columns=["mid", "hire_date"])
cprr

,badge_no,sex,department_desc,rank_desc,rule_code,paragraph_code,recommended_action,action,incident_type,tracking_number,...,occur_day,receive_year,receive_month,receive_day,investigation_complete_year,investigation_complete_month,investigation_complete_day,agency,data_production_year,uid
0,HP-56,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2015-3,...,5,2015,7,9,2015,8,4,New Orleans Harbor PD,2020,cac9967e392f4edf4da41c4e478f96b5
1,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2015-2,...,22,2015,4,1,2015,6,29,New Orleans Harbor PD,2020,2e448f6aba55cd31091991480f1af231
2,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2016-3,...,25,2016,6,27,2016,7,29,New Orleans Harbor PD,2020,2e448f6aba55cd31091991480f1af231
3,HP-50,male,off-duty detail,police officer 2-a,2,3,none,,discourtesy,2016-5,...,20,2016,12,21,2017,1,11,New Orleans Harbor PD,2020,2e448f6aba55cd31091991480f1af231
4,HP-33,male,bicycle division,corporal,1.3,2,none,,use of force,2017-7,...,15,2017,7,15,2017,8,29,New Orleans Harbor PD,2020,e6278fa24d4f3a03c7b1fc41f07dd6de
5,HP-56,male,,police officer 2-a,2,3,counselling memorrandum,counselling memmorandum,discourtesy,2017-1,...,24,2017,1,10,2017,2,20,New Orleans Harbor PD,2020,cac9967e392f4edf4da41c4e478f96b5
